In [1]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=fbaa16a3fe2a43528b0172ec4171a3ac48b906b6aae813c30203acac388511ed
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json

In [3]:
path = '/content/drive/MyDrive/HN/M2/phil_comp/odil-data-master-hn/data/translated_terms.json'
with open(path, 'r') as f:
  terms_dict = json.load(f)

# Langdetect

In [21]:
import pandas as pd
df = pd.DataFrame(terms_dict.items(), columns=['term', 'translation'])

In [12]:
from langdetect import detect
from langdetect import DetectorFactory
DetectorFactory.seed = 0

df['lang_original'] = df['term'].apply(lambda x: detect(x))
df['lang_translation'] = df['translation'].apply(lambda x: detect(x))

In [17]:
df['lang_original'].value_counts().reset_index().iloc[0],df['lang_translation'].value_counts().reset_index().iloc[0]

(lang_original      fr
 count            1038
 Name: 0, dtype: object,
 lang_translation     en
 count               848
 Name: 0, dtype: object)

FAISS

In [10]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 74.0 MB/s eta 0:00:00


In [5]:
!pip install tensorflow tensorflow-text

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [22]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import faiss
import tensorflow_text

# Carica Universal Sentence Encoder Multilingue
model_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
embed = hub.load(model_url)

In [23]:
fr_embeddings = embed(df['term'].tolist()).numpy().astype(np.float32)
en_embeddings = embed(df['translation'].tolist()).numpy().astype(np.float32)

In [25]:
dim = fr_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
# Aggiungi i target (inglese) all'indice
index.add(en_embeddings)

# Query: per ogni parola francese, trova il match inglese
D, I = index.search(fr_embeddings, k=1)  # k=1: top-1 similarità

In [30]:
df['similarity'] = D.flatten()

In [39]:
df['similarity'].describe()

,similarity
count,1971.000000
mean,0.802534
std,0.123650
min,0.426335
25%,0.706423
50%,0.803503
75%,0.901840
max,1.000000
